In [1]:
# Import libraries
import numpy as np
import geopandas as gpd
import momepy
import time
import copy
import networkx as nx
# import pandas as pd
# import shapely
# import shapely.geometry as sg
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline

from lmzintgraf_gp_pref_elicit import dataset, gaussian_process, acquisition_function
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_ccs as utils_ccs
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_data as utils_data
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_experiment as utils_experiment
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_parameters as utils_parameters
from lmzintgraf_gp_pref_elicit.gp_utilities import utils_user as utils_user

In [2]:
map_amsterdam = gpd.read_file("Sidewalk_width_crossings_smaller.geojson") #Read in the map with radius 250m and ~1000 nodes

# Objectives
objective1 = map_amsterdam['length']
objective2 = map_amsterdam['crossing']
objective3 = map_amsterdam['obstacle_free_width']

objectives = ('length', 'crossing')

In [3]:
# Create a NetworkX graph from the map
G = momepy.gdf_to_nx(map_amsterdam, approach='primal')
nodes = G.nodes
edges = G.edges
# print(G)

/usr/local/anaconda3/envs/test/lib/python3.10/site-packages/momepy/utils.py:252: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf_network[length] = gdf_network.geometry.length


In [ ]:
# print(nodes)

In [4]:
#Pick random ones or pick manually that make sense - to experiment
S = (122245.37633330293, 486126.8581684635) #very first node
T = (122320.31466476223, 486327.5294561802)
# T = (122253.09793657108, 486219.18429932056)

In [5]:
def pareto_dominates(a, b):
    """Check if the vector in b Pareto dominates vector a.

    Note: The original code has been modified to work for our minimization problem.

    Args:
        a (ndarray): A numpy array.
        b (ndarray): A numpy array.

    Returns:
        bool: Whether vector b dominates vector a.
    """
    a = np.array(a)
    b = np.array(b)
    return np.all(a <= b) and np.any(a < b)


def p_prune(candidates):
    """Create a Pareto coverage set from a set of candidate points.

    References:
        .. [1] Roijers, D. M., & Whiteson, S. (2017). Multi-objective decision making. 34, 129–129.
            https://doi.org/10.2200/S00765ED1V01Y201704AIM034

    Args:
        candidates (Set[Tuple]): A set of vectors.

    Returns:
        Set[Tuple]: A Pareto coverage set.
    """
    pcs = set()
    while candidates:
        vector = candidates.pop()

        for alternative in candidates:
            if pareto_dominates(alternative, vector):
                vector = alternative

        to_remove = set(vector)
        for alternative in candidates:
            if pareto_dominates(vector, alternative):
                to_remove.add(alternative)

        candidates -= to_remove
        pcs.add(vector)
    return pcs


def pvi(G, T, objectives, max_iter=100):
    start = time.time()
    nd_vectors = [[{tuple(np.full(2, np.inf))} for _ in range(len(G.nodes))] for _ in range(len(G.nodes))] # Initialisation of nodes
    is_last = False
    converged = False

    for n, current_node in enumerate(G.nodes):
        if current_node == T:  # We've reached the terminal state
            nd_vectors[n] = [{(0, 0)} for _ in G.nodes]
            # converged = True
            break

    nd_vectors_update = copy.deepcopy(nd_vectors)
    # for run in range(max_iter):  # We execute the algorithm for a number of iterations.
        # print(f'Value Iteration number: {run}')

    # while '{(inf, inf)}' in str(nd_vectors_update):
    converged = False
    while not converged:  # Run until convergence
        converged = True

        for n, current_node in enumerate(G.nodes):  # Loop over all states. Note: current_node is an object; n=number
            if current_node == T:
                continue

            for nk, neighbor in enumerate(G.nodes): #Note: neighbor is an object; k=number
                if neighbor not in G.neighbors(current_node):
                    continue

                edge = G[current_node][neighbor]
                edge_list = [v for k, v in edge.items()]  # Stores only the values of the edges' properties

                cost = []
                for i in objectives:
                  cost.append(edge_list[0][i])

                cost = np.array(cost)
                results = set()

                for v_list in nd_vectors[nk]:
                    for value_vec in v_list:
                        results.add(tuple(cost+value_vec)) # The set of candidate vectors

                results = p_prune(results)

                if nd_vectors_update[n][nk] != results:
                    converged = False
                    nd_vectors_update[n][nk] = results

        if converged:
            break

        nd_vectors = copy.deepcopy(nd_vectors_update)  # Else perform a deep copy and go again.


    end = time.time()
    elapsed_seconds = (end - start)
    print("Seconds elapsed: " + str(elapsed_seconds))

    return nd_vectors_update



pvi_result = pvi(G, T, objectives)
print(pvi_result)


Seconds elapsed: 23.337018966674805
[[{(inf, inf)}, {(284.05000000000007, 4.0)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, inf)}, {(inf, in

In [ ]:
'{(inf, inf)}' in str(pvi_result)

In [ ]:
# for each action in A:
# retireve R=cost
# Q is set of value vector
# s' is neighbor
# T is 1
# actions = neighbors


def track_policy(G, T, target, pvi_vec, tol=1e-3):

        terminated = False
        truncated = False
        reward = np.zeros(2)


        while not (terminated or truncated):
            closest_dist = np.inf
            closest_action = 0
            found_action = False
            new_target = target

            for n, current_node in enumerate(G.nodes):
                if current_node == T:
                    terminated = True
                    truncated = True
                else:
                    for nk, neighbor in enumerate(G.neighbors(current_node)):
                        im_rew = [n, nk]
                        # print(f"im_rew:{im_rew}")
                        non_dominated_set = pvi_vec[n][nk]
                        # print(f"non_dominated set:{non_dominated_set}")

                        for q in non_dominated_set:
                            q = np.array(q)
                            # print(f"q:{q}")
                            dist = np.sum(np.abs(q + im_rew - target))
                            # print(f"distance:{dist}")
                            if dist < closest_dist:
                                closest_dist = dist
                                closest_action = nk
                                new_target = q

                                if dist < tol:
                                    found_action = True
                                    break

                        if found_action:
                            break

                #closest action = edge you want to take
                #step = go to neighbor node
                closest_action = n
                reward = im_rew
                target = new_target



        return reward

# def construct_paths(S, T, path_dict):
#     paths = []
#     current_node = T
#
#     while current_node != S:
#         prev_node = path_dict[current_node]
#         path = [prev_node, current_node]
#         paths.append(path)
#         current_node = prev_node
#
#     paths.reverse()  # Reverse the paths to get the correct order from S to T
#
#     return paths

res = track_policy(G, T, (157.33,2.), pvi_result)
print(res)

In [ ]:
# def track_policy():
#         """Track a policy from its return vector.
#
#         Args:
#             vec (array_like): The return vector to track.
#             G: Graph
#             tol (float, optional): The tolerance for the return vector. (Default value = 1e-3)
#         """
#         # target = np.array(vec)
#         # state, _ = env.reset()
#         terminated = False
#         truncated = False
#         total_rew = np.zeros(self.num_objectives)
#
#         while not (terminated or truncated):
#             # state = np.ravel_multi_index(state, self.env_shape)
#             closest_dist = np.inf
#             closest_action = 0
#             found_action = False
#             new_target = target
#
#             for n, current_node in enumerate(G.nodes):
#                 for nk, neighbor in enumerate(G.neighbors(current_node)):
#                     im_rew = self.avg_reward[n, nk]
#                     non_dominated_set = self.non_dominated[n][nk]
#
#                     for q in non_dominated_set:
#                         q = np.array(q)
#                         dist = np.sum(np.abs(q + im_rew - t))
#                         if dist < closest_dist:
#                             closest_dist = dist
#                             closest_action = nk
#                             new_target = q
#
#                             if dist < tol:
#                                 found_action = True
#                                 break
#
#                     if found_action:
#                         break
#
#                 #closest action = edge you want to take
#                 #step = go to neighbor node
#                 state, reward, terminated, truncated, _ = G.nodes(closest_action) #go to the node that it's at the end of the edge
#                 total_rew += reward
#                 target = new_target
#
#         return total_rew
